<a href="https://colab.research.google.com/github/Nathalia-git/Aplicacao-de-XAI-para-o-design-de-farmacos/blob/main/TCC_Peptideo_Antimicrobiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalando bibliotecas necessárias**

## **Instalando conda**

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

In [ ]:
!conda --version

## **Baixando e instalando a biblioteca Pfeature**



In [ ]:
! wget https://github.com/raghavagps/Pfeature/raw/master/PyLib/Pfeature.zip

In [ ]:
! unzip Pfeature.zip

In [ ]:
cd /content/Pfeature

In [ ]:
! python setup.py install

## **Instalando CD-HIT**

In [ ]:
! conda install -c bioconda cd-hit -y

# **Carregando o conjunto de dados**


In [ ]:
! wget https://raw.githubusercontent.com/dataprofessor/AMP/main/train_po.fasta

In [ ]:
! wget https://raw.githubusercontent.com/dataprofessor/AMP/main/train_ne.fasta

In [ ]:
! cat train_ne.fasta

### **Removendo sequências redundantes usando CD-HIT**

In [ ]:
! cd-hit -i train_po.fasta -o train_po_cdhit.txt -c 0.99

In [ ]:
! cd-hit -i train_ne.fasta -o train_ne_cdhit.txt -c 0.99

In [ ]:
! ls -l

In [ ]:
#visualizando o numero de linhas no arquivo apos a retirada dos peptideos semelhantes
! grep ">" train_po_cdhit.txt | wc -l

In [ ]:
! grep ">" train_po.fasta | wc -l

In [ ]:
! grep ">" train_ne.fasta | wc -l

In [ ]:
#visualizando o numero de linhas no arquivo apos a retirada dos peptideos semelhantes
! grep ">" train_ne_cdhit.txt | wc -l

# **Calculando recursos usando a biblioteca Pfeature**

As classes das features fornecidas pelo Pfeature que estão sendo utilizadas estão resumidas na tabela abaixo.

**Recursos Baseados em Composição**

Classe de Recursos | Descrição | Função
---|---|---
AAC | Composição de Aminoácidos | aac_wp
DPC | Composição Dipeptídeo | dpc_wp
[Pfeature Manual](https://webs.iiitd.edu.in/raghava/pfeature/Pfeature_Manual.pdf)

### **Funções para calcular diferentes recursos**

In [ ]:
import pandas as pd

In [ ]:
#Composicao de Aminoacidos (AAC)

from Pfeature.pfeature import aac_wp

def aac(input):
  z = input.rstrip('txt')
  output = z + 'aac.csv'
  aac = aac_wp(input, output)
  aac_in_csv = pd.read_csv(output)
  return aac_in_csv

feature_aac_pos = aac('train_po_cdhit.txt')
feature_aac_neg = aac('train_ne_cdhit.txt')

feature_aac_pos
feature_aac_neg

In [ ]:
#Composicao Dipeptideo (DPC)

from Pfeature.pfeature import dpc_wp

def dpc(input):
  z = input.rstrip('txt')
  output = z + 'dpc.csv'
  dpc = dpc_wp(input, output,1)
  dpc_in_csv = pd.read_csv(output)
  return dpc_in_csv

feature_dpc_pos = dpc('train_po_cdhit.txt')
feature_dpc_neg = dpc('train_ne_cdhit.txt')

feature_dpc_pos
feature_dpc_neg

### **Combinando diferentes recursos**

In [ ]:
#positivo
all_feature_pos = pd.concat([feature_aac_pos, feature_dpc_pos], axis=1)
all_feature_pos

In [ ]:
#negativo
all_feature_neg = pd.concat([feature_aac_neg, feature_dpc_neg], axis=1)
all_feature_neg

### **Calculando o recurso para a classe positiva e negativa, combinando as duas classes e mesclando com os rótulos das classes**

In [ ]:
pos = 'train_po_cdhit.txt'
neg = 'train_ne_cdhit.txt'

def feature_calc(po, ne, all_feature_pos, all_feature_neg):
  # Calcula recurso
  positive_feature = all_feature_pos
  negative_feature = all_feature_neg
  # Cria rotulos de classe
  positive_class = pd.Series(['positive' for i in range(len(positive_feature))])
  negative_class = pd.Series(['negative' for i in range(len(negative_feature))])
  # Combina positivo e negativo
  positive_negative_class = pd.concat([positive_class, negative_class], axis=0)
  positive_negative_class.name = 'class'
  positive_negative_feature = pd.concat([positive_feature, negative_feature], axis=0)
  # Combina recurso e classe
  df = pd.concat([positive_negative_feature, positive_negative_class], axis=1)
  return df

all_feature = feature_calc(pos, neg, all_feature_pos, all_feature_neg)

In [ ]:
all_feature

In [ ]:
# Atribui as features em X e os rótulos das classes em y
X = all_feature.drop('class', axis=1) #os valores sao passados para x sem a coluna de classe
y = all_feature['class'].copy() #é copiado para y apenas a coluna de classe que define se positivo ou negativo

In [ ]:
# Encoding the Y class label
y = y.map({"positive": 1, "negative": 0})  #criado um mapa de positivo e negativo para y porque ele atribui a coluna de classe

In [ ]:
y

In [ ]:
X.shape

In [ ]:
y.shape

### **Removendo recursos com baixa variação no conjunto de dados**

In [ ]:
from sklearn.feature_selection import VarianceThreshold

low_variation = VarianceThreshold(threshold=0.1)
low_variation.fit_transform(X)
X_new = X.loc[:, low_variation.get_support()]

X_new

# **Dividindo o conjunto de dados em treino e teste**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state =42, stratify=y)

### **Selecionando os melhores recursos para a classificação e determinando a importância de cada recurso por meio da validação cruzada**

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

# A pontuação da "acuracia" é proporcional ao número de classificações corretas
clf_rf = RandomForestClassifier()
rfecv = RFECV(estimator=clf_rf, step=1, cv=5,scoring='accuracy')   #5-fold cross-validation
rfecv = rfecv.fit(X_train, y_train)


In [ ]:
rfecv.n_features_ #numero otimo de recursos

In [ ]:
X_train.columns[rfecv.support_] #melhores features

In [ ]:
X_train_2 = X_train.loc[:, X_train.columns[rfecv.support_]] #criando um novo dataframe contendo apenas as melhores features

In [ ]:
X_train_2

In [ ]:
#calculando a acuracia e a matriz de confusao usando o classificador RandomForestClassifier com as melhores features
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,confusion_matrix
import seaborn as sns

X_train_2 = X_train.loc[:, X_train.columns[rfecv.support_]] #cria novo dataframe X_train_2 contendo apenas as melhores features selecionadas no conjunto de treinamento
X_test_2 = X_test.loc[:, X_test.columns[rfecv.support_]] #cria novo dataframe X_test_2 contendo apenas as melhores features selecionadas no conjunto de teste
#random forest classifier with n_estimators=10 (default)
clf_rf_2 = RandomForestClassifier()
clr_rf_2 = clf_rf_2.fit(X_train_2,y_train)#Ajustando o modelo RFE e ajustando automaticamente o número de recursos selecionados.
ac_2 = accuracy_score(y_test,clf_rf_2.predict(X_test_2)) #calcula acuracia.  A acuracia é medida comparando as previsões do modelo com os rótulos y_test verdadeiros.
cm_2 = confusion_matrix(y_test,clf_rf_2.predict(X_test_2)) #calcula matriz de confusao. A matriz de confusão é uma tabela que mostra as classificações corretas e incorretas feitas pelo modelo.
sns.heatmap(cm_2,annot=True,fmt="d") #plota a matriz de confusao

In [ ]:
ac_2 #acuracia

# **Comparando rapidamente mais de 30 algoritmos de Machine Learning**

In [ ]:
!pip install lazypredict

In [ ]:
#treinando e avaliando vários modelos de classificação de forma rápida e automática.

from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
import lazypredict

# Define e constrói o lazyclassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=matthews_corrcoef)
models_lazy_train,predictions_train = clf.fit(X_train_2, X_train_2, y_train, y_train)
models_lazy_test,predictions_test = clf.fit(X_train_2, X_test_2, y_train, y_test)

In [ ]:
#Imprime o desempenho do modelo para o conjunto de treinamento
models_lazy_train

In [ ]:
#Imprime o desempenho do modelo para o conjunto de teste
models_lazy_test

In [ ]:
#Plota Acuracia
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(y=models_lazy_train.index, x="Accuracy", data=models_lazy_train)
ax.set(xlim=(0, 1))

In [ ]:
# Plota MCC
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(y=models_lazy_train.index, x="matthews_corrcoef", data=models_lazy_train)
ax.set(xlim=(0, 1))

# **Random Forest**

In [ ]:
#Construindo o modelo Random Forest

from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=500)

random_forest.fit(X_train_2, y_train)

### **Aplicando o modelo para fazer previsões**

In [ ]:
y_train_predict = random_forest.predict(X_train_2)
y_test_predict = random_forest.predict(X_test_2)

In [ ]:
y_train_predict

In [ ]:
X_train_2.shape
#y_test

In [ ]:
y_test_predict

### **Desempenho do modelo**

In [ ]:
# Simplest and quickest way to obtain the model performance (Accuracy)
random_forest.score(X_test_2,y_test)

In [ ]:
# Accuracy
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_test_predict)

In [ ]:
# Matthew Correlation Coefficient
from sklearn.metrics import matthews_corrcoef

matthews_corrcoef(y_test, y_test_predict)

In [ ]:
cm = confusion_matrix(y_test, y_test_predict)
sns.heatmap(cm,annot=True,fmt="d")

In [ ]:
# Classification report
from sklearn.metrics import classification_report

model_report = classification_report(y_train, y_train_predict, target_names=['positive','negative'])

f = open('model_report.txt','w')
f.writelines(model_report)
f.close()

In [ ]:
# ROC curve
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

##RocCurveDisplay.from_estimator(svc, X_test, y_test)
RocCurveDisplay.from_estimator(clf_rf_2, X_test_2, y_test)

##plot_roc_curve(random_forest, X_test, y_test)
plt.show()

In [ ]:
RocCurveDisplay.from_estimator(random_forest, X_train_2, y_train)
plt.show()

### **Importância do recurso**

In [ ]:
# Recupera a importancia da feature do modelo RF
importance = pd.Series(random_forest.feature_importances_, name = 'Gini')

# Recupera nomes das features
feature_names = pd.Series(X_train_2.columns, name = 'Feature')

In [ ]:
# Combina nomes das features e valores Gini em um dataframe
dataframe = pd.concat([feature_names, importance], axis=1, names=['Feature', 'Gini'])
dataframe

In [ ]:
# Grafico de importancia da feature
import matplotlib.pyplot as plt
import seaborn as sns

dataframe_sorted = dataframe.sort_values('Gini', ascending=False)[:20] # Sort by Gini in descending order; Showing only the top 20 results

plt.figure(figsize=(5, 10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(x = 'Gini', y = 'Feature', data = dataframe_sorted)
plt.xlabel("Feature Importance")

### **Grafico de importância do recurso com RF**




In [ ]:
#o pipeline padroniza os recursos usando o StandardScaler e em seguida treina um modelo de Random Forest Classifier usando os dados de treinamento fornecidos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder

random_forest_pipeline = Pipeline(steps = [('scale',StandardScaler()),('RF',RandomForestClassifier(random_state=42))])
random_forest_pipeline.fit(X_train_2,y_train)

In [ ]:
#calculando e visualizando a importância das features (variáveis independentes) no modelo de Random Forest.
import matplotlib

colors = ["lightgray","lightgray","#0f4c81"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

def rf_feat_importance(m, X):
    return pd.DataFrame({'Feature':X_train_2.columns.tolist(), 'Importance':m.feature_importances_}).sort_values('Importance', ascending=False)


fi = rf_feat_importance(random_forest_pipeline['RF'], X_train_2)
fi[:20].style.background_gradient(cmap=colormap)

#**XAI**

## **Aplicando SHAP para os dados de treinamento**

In [ ]:
!conda install --upgrade numba numpy

In [ ]:
!conda install -c conda-forge shap

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(random_forest)

In [ ]:
shap_values = explainer.shap_values(X_train_2)
X_train_2

In [ ]:
shap.summary_plot(shap_values[1], X_train_2)

In [ ]:
shap.summary_plot(shap_values[1], X_train_2, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values[1], X_train_2, plot_type="violin")

## **Utilizando os peptídeos desenvolvidos por pesquisadores da UCB para validação**



In [ ]:
validation_feat_aac = aac('/content/val_pos_teste.txt')
validation_feat_dpc = dpc('/content/val_pos_teste.txt')

In [ ]:
all_test_feat = pd.concat([validation_feat_aac, validation_feat_dpc], axis=1)
all_test_feat

In [ ]:
X24 = all_test_feat.loc[:, X_train.columns[rfecv.support_]]

In [ ]:
X24

In [ ]:
validacao = random_forest.predict(X24)

In [ ]:
validacao

## **Aplicando SHAP para os dados de validação**

In [ ]:
!pip install shap
import shap

In [ ]:
explainer = shap.TreeExplainer(random_forest)

In [ ]:
shap_values = explainer.shap_values(X24)

In [ ]:
shap.summary_plot(shap_values[1], X24, plot_type="violin")

In [ ]:
# plotando para a classe positiva
shap.summary_plot(shap_values[1], X24)

In [ ]:
shap.summary_plot(shap_values[1], X24, plot_type="bar")

In [ ]:
shap.dependence_plot('AAC_K', shap_values[1], X24, interaction_index="AAC_D")

In [ ]:
shap.dependence_plot('AAC_K', shap_values[1], X24, interaction_index=None)

In [ ]:
row_to_show = 0

data_for_prediction = X24.iloc[row_to_show]
data_for_prediction_array = data_for_prediction.values.reshape(1, -1)

random_forest.predict_proba(data_for_prediction_array)

In [ ]:
shap_values2 = explainer.shap_values(data_for_prediction)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values2[1], data_for_prediction)

In [ ]:
shap.initjs()
shap_values = explainer.shap_values(X24.iloc[:50])
shap.force_plot(explainer.expected_value[1], shap_values[1], X24.iloc[:50])

## **Aplicando LIME para os dados de validação**

In [ ]:
! pip install lime

In [ ]:
import lime
import lime.lime_tabular
import random

In [ ]:
#criando explainer
predict_fn_rf = lambda x: random_forest.predict_proba(x).astype(float)
x = X24.values
explainer = lime.lime_tabular.LimeTabularExplainer(x,feature_names = X24.columns,class_names=['N_AMP','AMP'],kernel_width=5)

In [ ]:
random_forest.predict(X24)

In [ ]:
random_forest.predict_proba(x).astype(float)

In [ ]:
row_to_show =0

In [ ]:
choosen_instance = X24.loc[[row_to_show]].values[0]

exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=12)
exp.show_in_notebook(show_all=False)

In [ ]:
exp.as_list()

In [ ]:
with plt.style.context("ggplot"):
    exp.as_pyplot_figure()